In [2]:
import re
import numpy as np
import pandas as pd

CSV_PATH = "/francetravail.csv"
df = pd.read_csv(CSV_PATH)

# 1) Fusion propre du texte salaire
sal_lib = [c for c in df.columns if c.lower().replace('_','.') == 'salaire.libelle']
col_lib = sal_lib[0] if sal_lib else 'salaire.libelle'

def detect_type(txt) -> str:
    t = str(txt).lower().strip()
    # Priorité forte : horaire > annuel > mensuel
    if re.search(r'\b(horaire)\b', t):
        return 'horaire'
    if re.search(r'\b(annuel(le)?)\b', t):
        return 'annuel'
    if re.search(r'\b(mensuel)\b', t):
        return 'mensuel'
    return 'autre'

df['salaire_type'] = df[col_lib].apply(detect_type)

# 2) Extraction des montants MONÉTAIRES (on ignore les nombres sans unité monétaire)
# - prend 35k, 35 k, 35k€, 2 500 €, 2500 euros, 45 K EUR, etc.
amount_pattern = re.compile(
    r'(?P<val>\d[\d\s.,]*)(?P<unit>\s*[kK])?\s*(?P<currency>€|euros?|eur)?'
)

def extract_money_values(txt):
    txt = str(txt).lower().strip()
    vals = []
    for m in amount_pattern.finditer(txt):
        cur = m.group('currency')
        unitk = m.group('unit')
        raw = m.group('val')

        # on accepte seulement si une "monnaie" est là (€, euro, eur) ou un 'k' explicite (k€)
        # -> ainsi, "12 mois" (sans €) n'est pas retenu
        if not cur and not unitk:
            continue

        # nettoyage nombre
        v = raw.replace(' ', '').replace('\u202f','').replace(',', '.')
        try:
            v = float(v)
        except:
            continue

        # gestion k / K = milliers
        if unitk:
            v *= 1000.0

        # quelques faux positifs possibles (ex: "2 k ms") sont éliminés plus tard par bornage
        vals.append(v)

    return vals

df['money_vals'] = df[col_lib].apply(extract_money_values)

# 3) Sélection min/max/moyenne contextuelle
# Pour HORAIRE, on garde aussi les petites valeurs (ex: 12.5 €/h).
# Pour les autres, la contrainte de "unité monétaire" a déjà évité 12/13 de "mois".
def pick_min(vals):
    return float(vals[0]) if len(vals) >= 1 else np.nan

def pick_max(vals):
    return float(vals[1]) if len(vals) >= 2 else np.nan

df['salaire_min'] = df['money_vals'].apply(pick_min)
df['salaire_max'] = df['money_vals'].apply(pick_max)
df['salaire_moy'] = np.nanmean(df[['salaire_min','salaire_max']], axis=1)

# 4) Conversion en annuel (priorité du type conservée)
def to_annual(row):
    v = row['salaire_moy']
    t = row['salaire_type']
    if pd.isna(v):
        return np.nan
    if t == 'horaire':
        return v * 35 * 52
    if t == 'mensuel':
        return v * 12
    if t == 'annuel':
        return v
    # "autre" -> heuristique douce : si v <= 100 -> suppose horaire, si 100< v < 10000 -> mensuel, sinon annuel
    if v <= 100:
        return v * 35 * 52
    if v < 10000:
        return v * 12
    return v

df['salaire_annuel_estime'] = df.apply(to_annual, axis=1)

# 5) Bornage anti-aberrations (à ajuster si besoin)
df.loc[(df['salaire_annuel_estime'] < 6000) | (df['salaire_annuel_estime'] > 200000), 'salaire_annuel_estime'] = np.nan

# 6) Contrôle rapide
cols = [c for c in [col_lib] if c in df.columns] + ['salaire_type','salaire_min','salaire_max','salaire_annuel_estime']
print(df[cols].head(12))

# 7) Sauvegarde
df.to_csv("francetravail_salaire_normalise.csv", index=False)
print("OK → francetravail_salaire_normalise.csv écrit.")

/tmp/ipython-input-610980919.py:73: RuntimeWarning: Mean of empty slice
  df['salaire_moy'] = np.nanmean(df[['salaire_min','salaire_max']], axis=1)


                                      salaire.libelle salaire_type  \
0               Annuel de 30000.0 Euros sur 12.0 mois       annuel   
1                              Horaire de 11.65 Euros      horaire   
2               Annuel de 50000.0 Euros sur 12.0 mois       annuel   
3   Mensuel de 2059.00 Euros à 3000.00 Euros sur 1...      mensuel   
4               Mensuel de 2500.0 Euros sur 12.0 mois      mensuel   
5               Annuel de 40000.0 Euros sur 12.0 mois       annuel   
6   Horaire de 11.88 Euros à 12.0 Euros sur 12.0 mois      horaire   
7               Annuel de 45000.0 Euros sur 12.0 mois       annuel   
8   Mensuel de 32000.0 Euros à 45000.0 Euros sur 1...      mensuel   
9   Annuel de 32000.0 Euros à 40000.0 Euros sur 12...       annuel   
10  Mensuel de 2500.0 Euros à 3000.0 Euros sur 12....      mensuel   
11              Annuel de 38000.0 Euros sur 12.0 mois       annuel   

    salaire_min  salaire_max  salaire_annuel_estime  
0      30000.00          NaN       